In [ ]:
import math
import sys
import numpy as np
np.set_printoptions(threshold=sys.maxsize)
import pandas as pd
from scipy import linalg
from tabulate import tabulate

In [ ]:
from google.colab import drive
drive.mount ('/content/drive', force_remount=True)

Mounted at /content/drive


PREPROCESSING DATA
===============================================

In [ ]:
PATH = '/content/drive/My Drive/Colab Notebooks/dataset/projectDSSv2.csv'
df = pd.read_csv(PATH)
df

In [ ]:
PATH = '/content/drive/My Drive/Colab Notebooks/dataset/ProjectDSS.csv'
df = pd.read_csv(PATH)
df

,Provinsi,Kepadatan Penduduk (jiwa/km2),Jumlah Penduduk Miskin (Ribu Jiwa),Persentase Penduduk yang Mempunyai Keluhan Kesehatan Selama Sebulan Terakhir (Persen),"Jumlah Mahasiswa(Negeri dan Swasta) di Bawah Kementerian Riset, Teknologi dan Pendidikan Tinggi/Kementerian Pendidikan dan Kebudayaan Menurut Provinsi,",Pekerja Mover (Komuter),Persentase Penduduk Lansia
0,DKI JAKARTA,15900,496.84,33.80,707707,1035784,8.21
1,JAWA BARAT,1394,4188.52,32.04,751785,2312831,9.59
2,JAWA TENGAH,1058,4119.93,35.63,572837,1092459,13.81
3,DI YOGYAKARTA,1227,503.14,38.07,369831,277830,14.71
4,JAWA TIMUR,831,4585.97,32.80,818111,1038899,13.38
5,BANTEN,1338,857.64,32.22,1308214,795314,6.60


In [ ]:
kriteria = np.array ([
              'kepadatan_penduduk',
              'jumlah_penduduk_miskin',
              'persentase_keluhan_penyakit',
              'jumlah_mahasiswa',
              'jumlah_pekerja_komuter',
              'persentase_penduduk_lansia'
])

indexing_C = pd.Series(kriteria, index = ['C1', 'C2', 'C3', 'C4', 'C5', 'C6'])
print(indexing_C)

C1             kepadatan_penduduk
C2         jumlah_penduduk_miskin
C3    persentase_keluhan_penyakit
C4               jumlah_mahasiswa
C5         jumlah_pekerja_komuter
C6     persentase_penduduk_lansia
dtype: object


In [ ]:
alternatif = np.array ([
                        'DKI JAKARTA',
                        'JAWA BARAT',
                        'JAWA TENGAH',
                        'DI YOGYAKARTA',
                        'JAWA TIMUR',
                        'BANTEN'
])

indexing_A = pd.Series(alternatif, index = ['A1','A2','A3','A4','A5','A6'])
print(indexing_A)

A1      DKI JAKARTA
A2       JAWA BARAT
A3      JAWA TENGAH
A4    DI YOGYAKARTA
A5       JAWA TIMUR
A6           BANTEN
dtype: object


In [ ]:
alternatif = np.array ([
                        'ACEH',
                        'SUMATERA UTARA',
                        'SUMATERA BARAT',
                        'RIAU',
                        'JAMBI',
                        'SUMATERA SELATAN',
                        'BENGKULU',
                        'LAMPUNG',
                        'KEP. BANGKA BELITUNG',
                        'KEP. RIAU',
                        'DKI JAKARTA',
                        'JAWA BARAT',
                        'JAWA TENGAH',
                        'DI YOGYAKARTA',
                        'JAWA TIMUR',
                        'BANTEN',
                        'BALI',
                        'NUSA TENGGARA BARAT',
                        'NUSA TENGGARA TIMUR',
                        'KALIMANTAN BARAT',
                        'KALIMANTAN TENGAH',
                        'KALIMANTAN SELATAN',
                        'KALIMANTAN TIMUR',
                        'KALIMANTAN UTARA',
                        'SULAWESI UTARA',
                        'SULAWESI TENGAH',
                        'SULAWESI SELATAN',
                        'SULAWESI TENGGARA',
                        'GORONTALO',
                        'SULAWESI BARAT',
                        'MALUKU',
                        'MALUKU UTARA',
                        'PAPUA BARAT',
                        'PAPUA'
])

indexing_A = pd.Series(alternatif, index = ['A1','A2','A3','A4','A5','A6', 'A7','A8','A9','A10','A11','A12','A13','A14','A15','A16','A17','A18','A19','A20','A21','A22','A23','A24','A25','A26','A27','A28','A29','A30','A31','A32','A33','A34'])
print(indexing_A)

In [ ]:
df.columns = [
              'provinsi',
              'kepadatan_penduduk',
              'jumlah_penduduk_miskin',
              'persentase_keluhan_penyakit',
              'jumlah_mahasiswa',
              'jumlah_pekerja_komuter',
              'persentase_penduduk_lansia'
]
df

,provinsi,kepadatan_penduduk,jumlah_penduduk_miskin,persentase_keluhan_penyakit,jumlah_mahasiswa,jumlah_pekerja_komuter,persentase_penduduk_lansia
0,DKI JAKARTA,15900,496.84,33.80,707707,1035784,8.21
1,JAWA BARAT,1394,4188.52,32.04,751785,2312831,9.59
2,JAWA TENGAH,1058,4119.93,35.63,572837,1092459,13.81
3,DI YOGYAKARTA,1227,503.14,38.07,369831,277830,14.71
4,JAWA TIMUR,831,4585.97,32.80,818111,1038899,13.38
5,BANTEN,1338,857.64,32.22,1308214,795314,6.60


DATA YANG AKAN DIGUNAKAN

In [ ]:
df.columns = [
              'alternatif',
              'C1',
              'C2',
              'C3',
              'C4',
              'C5',
              'C6'
]

df['alternatif'] = ['A1', 'A2', 'A3', 'A4', 'A5', 'A6', 'A7','A8','A9','A10','A11','A12','A13','A14','A15','A16','A17','A18','A19','A20','A21','A22','A23','A24','A25','A26','A27','A28','A29','A30','A31','A32','A33','A34']
dfNew = df.set_index('alternatif')
dfNew

In [ ]:
df.columns = [
              'alternatif',
              'C1',
              'C2',
              'C3',
              'C4',
              'C5',
              'C6'
]

df['alternatif'] = ['A1', 'A2', 'A3', 'A4', 'A5', 'A6']
dfNew = df.set_index('alternatif')
dfNew

,C1,C2,C3,C4,C5,C6
alternatif,,,,,,
A1,15900,496.84,33.80,707707,1035784,8.21
A2,1394,4188.52,32.04,751785,2312831,9.59
A3,1058,4119.93,35.63,572837,1092459,13.81
A4,1227,503.14,38.07,369831,277830,14.71
A5,831,4585.97,32.80,818111,1038899,13.38
A6,1338,857.64,32.22,1308214,795314,6.60


PROSES FUZZY - MENENTUKAN WEIGHT MASING-MASING KRITERIA
===============================================

(1) LINGUISTIC SCALE

In [ ]:
print('9 RANGE\n')
table_range = [
               [1, '(1, 1, 2)'],
               [2, '(1, 2, 3)'],
               [3, '(2, 3, 4)'],
               [4, '(3, 4, 5)'],
               [5, '(4, 5, 6)'],
               [6, '(5, 6, 7)'],
               [7, '(6, 7, 8)'],
               [8, '(7, 8, 9)'],
               [9, '(8, 9, 10)'],
]

print(tabulate(table_range))

9 RANGE

-  ----------
1  (1, 1, 2)
2  (1, 2, 3)
3  (2, 3, 4)
4  (3, 4, 5)
5  (4, 5, 6)
6  (5, 6, 7)
7  (6, 7, 8)
8  (7, 8, 9)
9  (8, 9, 10)
-  ----------


In [ ]:
l_scale = {
    "Fuzzy Number" : [9, 8, 7, 6, 5, 4, 3, 2, 1],
    "Linguistic" : ['Perfect', ' Absolute', 'Very Good', 'Fairly Good', 'Good', 'Preferable', 'Not Bad', 'Weak Advantage', 'Equal']
}

#load l_scale into Dataframe
df_lScale = pd.DataFrame(l_scale)
print(df_lScale)

   Fuzzy Number      Linguistic
0             9         Perfect
1             8        Absolute
2             7       Very Good
3             6     Fairly Good
4             5            Good
5             4      Preferable
6             3         Not Bad
7             2  Weak Advantage
8             1           Equal


(2) PAIRWISE COMPARISON

DETERMINE BY EXPERT

In [ ]:
judge1 = np.array([
                   [[1,1,2], [6,7,8], [4,5,6], [3,4,5], [2,3,4], [1,2,3]],
                   [[1/8,1/7,1/6], [1,1,2], [1/3,1/2,1], [1/4,1/3,1/2], [1/5,1/4,1/3], [1/6,1/5,1/4]],
                   [[1/6,1/5,1/4], [1,2,3], [1,1,2], [1/3,1/2,1], [1/4,1/3,1/2], [1/5,1/4,1/3]],
                   [[1/5,1/4,1/3], [2,3,4], [1,2,3], [1,1,2], [1/3,1/2,1], [1/4,1/3,1/2]],
                   [[1/4,1/3,1/2], [3,4,5], [2,3,4], [1,2,3], [1,1,2], [1/3,1/2,1]],
                   [[1/3,1/2,1], [4,5,6], [3,4,5], [2,3,4], [1,2,3], [1,1,2]]
])

judge2 = np.array([
                   [[1,1,2], [3,4,5], [2,3,4], [4,5,6], [1/3,1/2,1], [1,2,3]],
                   [[1/5,1/4,1/3], [1,1,2], [1/3,1/2,1], [1,2,3], [1/6,1/5,1/4], [1/4,1/3,1/3]],
                   [[1/4,1/3,1/2], [1,2,3], [1,1,2], [2,3,4], [3,4,5], [1/3,1/2,1]],
                   [[1/6,1/5,1/4], [1/3,1/2,1], [1/4,1/3,1/2], [1,1,2], [1/8,1/7,1/6], [1/5,1/4,1/3]],
                   [[1,2,3], [4,5,6], [3,4,5], [6,7,8], [1,1,2], [2,3,4]],
                   [[1/3,1/2,1], [2,3,4], [1,2,3], [3,4,5], [1/4,1/3,1/2], [1,1,2]]
])

# judge3 = np.array([
#                    [[1,1,2], [6,7,8], [2,3,4], [5,6,7], [4,5,6], [1/5,1/3,1/2]],
#                    [[1/8,1/7,1/6], [1,1,2], [1/7,1/6,1/5], [1/4,1/3,1/2], [1/6,1/5,1/4], [1/10,1/9,1/8]],
#                    [[1/4,1/3,1/2], [5,6,7], [1,1,2], [4,5,6], [2,3,4], [1/6,1/5,1/4]],
#                    [[1/7,1/6,1/5], [2,3,4], [1/6,1/5,1/4], [1,1,2], [1/4,1/3,1/2], [1/8,1/7,1/6]],
#                    [[1/6,1/5,1/4], [4,5,6], [1/4,1/3,1/2], [2,3,4], [1,1,2], [1/7,1/6,1/5]],
#                    [[2,3,4], [8,9,10], [4,5,6], [6,7,8], [5,6,7], [1,1,2]]
# ])
# judge3 = np.array([
#                    [[1,1,2], [6,7,8], [1,2,3], [1/4,1/3,1/2], [1,1,2], [2,3,4]],
#                    [[1/8,1/7,1/6], [1,1,2], [7,8,9], [8,9,10], [1/3,1/2,1], [1/3,1/2,1]],
#                    [[1/3,1/2,1], [1/9,1/8,1/7], [1,1,2], [2,3,4], [6,7,8], [1/8,1/7,1/6]],
#                    [[2,3,4],[1/10,1/9,1/8], [1/4,1/3,1/2], [1,1,2], [1/10,1/9,1/8], [3,4,5]],
#                    [[1,1,2], [1,2,3], [1/8,1/7,1/6], [8,9,10], [1,1,2], [4,5,6]],
#                    [[1/4,1/3,1/2], [3,4,5], [6,7,8], [1/5,1/4,1/3], [1/6,1/5,1/4], [1,1,2]]
# ])

# judge4 = np.array([
#                    [[1,1,2],[2,3,4], [8,9,10], [5,6,7], [7,8,9], [6,7,8]],
#                    [[1/4,1/3,1/2], [1,1,2], [1/6,1/5,1/4], [1,2,3], [1/3,1/2,1], [1/4,1/3,1/2]],
#                    [[1/10,1/9,1/8], [4,5,6], [1,1,2], [3,4,5], [1/4,1/3,1/2], [1/3,1/2,1]],
#                    [[1/7,1/6,1/5], [1/3,1/2,1], [1/5,1/4,1/3], [1,1,2], [1,1,2], [3,4,5]],
#                    [[1/9,1/8,1/7], [1,2,3], [2,3,4], [1,1,2], [1,1,2], [4,5,6]],
#                    [[1/8,1/7,1/6], [2,3,4], [1,2,3], [1/5,1/4,1/3], [1/6,1/5,1/4], [1,1,2]]
# ])
# print(judge1)
# print(judge2)

(3) SYNTHETIC JUDGEMENT

In [ ]:
synJudge = judge1*judge2
print(synJudge)

[[[1.00000000e+00 1.00000000e+00 4.00000000e+00]
  [1.80000000e+01 2.80000000e+01 4.00000000e+01]
  [8.00000000e+00 1.50000000e+01 2.40000000e+01]
  [1.20000000e+01 2.00000000e+01 3.00000000e+01]
  [6.66666667e-01 1.50000000e+00 4.00000000e+00]
  [1.00000000e+00 4.00000000e+00 9.00000000e+00]]

 [[2.50000000e-02 3.57142857e-02 5.55555556e-02]
  [1.00000000e+00 1.00000000e+00 4.00000000e+00]
  [1.11111111e-01 2.50000000e-01 1.00000000e+00]
  [2.50000000e-01 6.66666667e-01 1.50000000e+00]
  [3.33333333e-02 5.00000000e-02 8.33333333e-02]
  [4.16666667e-02 6.66666667e-02 8.33333333e-02]]

 [[4.16666667e-02 6.66666667e-02 1.25000000e-01]
  [1.00000000e+00 4.00000000e+00 9.00000000e+00]
  [1.00000000e+00 1.00000000e+00 4.00000000e+00]
  [6.66666667e-01 1.50000000e+00 4.00000000e+00]
  [7.50000000e-01 1.33333333e+00 2.50000000e+00]
  [6.66666667e-02 1.25000000e-01 3.33333333e-01]]

 [[3.33333333e-02 5.00000000e-02 8.33333333e-02]
  [6.66666667e-01 1.50000000e+00 4.00000000e+00]
  [2.50000000e

(4) FUZZY WEIGHT

In [ ]:
resR = np.ones((6,3))
for i in range (6):
  for j in range (6):
    resR[i] = resR[i]*synJudge[i][j]
  print(f"r{(i+1)} : {resR[i]}")

r1 : [1.1520e+03 5.0400e+04 4.1472e+06]
r2 : [9.64506173e-07 1.98412698e-05 2.31481481e-03]
r3 : [1.38888889e-03 6.66666667e-02 1.50000000e+01]
r4 : [1.15740741e-05 2.97619048e-04 5.55555556e-02]
r5 : [7.200e+01 3.360e+03 3.456e+05]
r6 : [4.00e+00 2.40e+02 4.32e+04]


MENGHITUNG W

In [ ]:
addRtemp = np.zeros((1,3)) #jumlah r
addR = np.zeros((1,3)) #1/jumlah r
for i in range (6):
  addRtemp[0] = addRtemp[0]+resR[i]
for j in range (3):
  addR[0][j] = 1/addRtemp[0][j] 
  
print(f"sigma r = {addRtemp}")
print(f"1/sigma r = {addR}")

sigma r = [[1.22800140e+03 5.40000670e+04 4.53601506e+06]]
1/sigma r = [[8.14331318e-04 1.85184955e-05 2.20457822e-07]]


In [ ]:
wTNF = np.empty((6,3))
for i in range (6):
  wTNF[i] = resR[i]*addR
  print(f"W{i+1} : {wTNF[i]}")

W1 : [0.93810968 0.93333218 0.91428268]
W2 : [7.85427583e-10 3.67430467e-10 5.10319032e-10]
W3 : [1.13101572e-06 1.23456637e-06 3.30686733e-06]
W4 : [9.42513100e-09 5.51145701e-09 1.22476568e-08]
W5 : [0.05863185 0.06222215 0.07619022]
W6 : [0.00325733 0.00444444 0.00952378]


(4) CALCULATE BNP

In [ ]:
W = np.empty((6))
for i in range (6):
  W[i] = (((wTNF[i][2]-wTNF[i][0])+(wTNF[i][1]-wTNF[i][0]))/3 + wTNF[i][0])
print(W[0]+W[1]+W[2]+W[3]+W[4]+W[5])

1.0


In [ ]:
print('WEIGH MASING-MASING KRITERIA\n------------------------------\n')
for i in range (6):
  print('W'+str(i)+' = '+str(W[i]))

WEIGH MASING-MASING KRITERIA
------------------------------

W0 = 0.9285748444577255
W1 = 5.54392360888141e-10
W2 = 1.8908164729505245e-06
W3 = 9.061414927052273e-09
W4 = 0.06568140773913983
W5 = 0.005741847370854471


TAHAP ELECTRE
===============================================
STEP 1: MATRIX NORMALIZATION

In [ ]:
def normalized_decision(data: [], root_arr: []):
  # , root_arr: []
    criteria = data
    root = []
    normal = []
    for i, row in enumerate(root_arr):
        tmp = 0
        for col in row:
            tmp += math.pow(col, 2)
        root.append(math.sqrt(tmp))

    for row in criteria:
        tmp = []
        for index, col in enumerate(row):
            res = col/root[index]
            tmp.append(res)
        normal.append(tmp)

    return np.array(normal)

STEP 2: COMPOSING WEIGTED

In [ ]:
def weighted_normalized(matrix: [], weight: []):
    result = []
    for row in matrix:
        tmp = []
        for i, col in enumerate(row):
            res = col * weight[i]
            tmp.append(res)
        result.append(tmp)
    return result

STEP 3: SET OF N INDEX OF CONCORDANCE AND DISCONDANCE

In [ ]:
def indexing_concordance(data: [], x: int, y: int):
    matrix = []
    tmp_a = data[x-1]
    tmp_b = data[y-1]
    for i, col in enumerate(tmp_a):
        res = 1 if col >= tmp_b[i] else 0
        matrix.append(res)
    return matrix
  
def indexing_discordance(data: [], x: int, y: int):
    matrix = []
    tmp_a = data[x - 1]
    tmp_b = data[y - 1]
    for i, col in enumerate(tmp_a):
        res = 1 if col < tmp_b[i] else 0
        matrix.append(res)
    return matrix

In [ ]:
def index_table(data: [], type: str):
    result = []
    for i, row1 in enumerate(data):
        for j, row2 in enumerate(data):
            x = i+1
            y = j+1
            if x is not y:
                if type is "concordance":
                    tmp: [] = indexing_concordance(data, x, y)
                    idt = (x, y)
                    result.append([idt, tmp])
                else:
                    tmp: [] = indexing_discordance(data, x, y)
                    idt = (x, y)
                    result.append([idt, tmp])
    return np.array(result)

STEP 4: CONCONRDANCE AND DISCONCORDANCE SUPREMACY

In [ ]:
def concordance(data: [], w: []):
    result = []
    for x, row1 in enumerate(data):
        tmp = []
        for y, row2 in enumerate(data):
            if x is y:
                tmp.append(0)
            else:
                tmp_arr = indexing_concordance(data, x+1, y+1)
                sum = 0
                for i, cell in enumerate(tmp_arr):
                    sum += 0 if cell is 0 else w[i]
                tmp.append(sum)
        result.append(tmp)
    return result

In [ ]:
def discordance(data: []):
    result = []
    tmp_res = []
    for x, row in enumerate(data):
        for y, row2 in enumerate(data):
            if x is y:
                tmp_res.append(0)
            elif x != y:
                tmp = []
                tmp2 = []
                for k in range(len(data[0])):
                    idx = indexing_discordance(data, x+1, y+1)
                    diff = abs(data[x][k] - data[y][k])
                    if idx[k] == 1:
                        disc = diff
                    else:
                        disc = 0
                    tmp.append(disc)
                    tmp2.append(diff)
                max1 = max(tmp)
                max2 = max(tmp2)
                res = max1/max2
                tmp_res.append(res)

    result = np.reshape(tmp_res, (-1, 6))
    return result

In [ ]:
def threshold(matrix: []):
    total = 0
    count = 0
    for row in matrix:
        for col in row:
            total += col
        count += 1
    return total / (count*(count-1))

STEP 5: TOTAL MATRIX

In [ ]:
def dominan_concordance(cor: [], threshold: float):
    tmp_res = []
    resultC = []
    for row in cor:
        tmp = []
        for col in row:
            x = 1 if col > threshold else 0
            tmp.append(x)
        tmp_res.append(tmp)
    resultC = np.array(tmp_res)
    
    return resultC

In [ ]:
def dominan_discordance(dis: [], threshold: float):
    tmp_res =[]
    resultD = []
    for row in dis:
        tmp = []
        for col in row:
            x = 1 if col > threshold else 0
            tmp.append(x)
        tmp_res.append(tmp)
    resultD = np.array(tmp_res)

    return resultD

METHOD MAIN

In [ ]:
def main():
    data_arr = dfNew.to_numpy()
    print(dfNew)
    weight = W
    data = np.array(data_arr)

    norm = normalized_decision(data, data.transpose())
    print("\n\nSTEP1: Normalized : \n")
    dfNorm = pd.DataFrame(norm, columns = ['C1', 'C2', 'C3', 'C4', 'C5', 'C6'], index = ['A1', 'A2', 'A3', 'A4', 'A5', 'A6', 'A7','A8','A9','A10','A11','A12','A13','A14','A15','A16','A17','A18','A19','A20','A21','A22','A23','A24','A25','A26','A27','A28','A29','A30','A31','A32','A33','A34'])
    print(dfNorm)

    norm_weight = weighted_normalized(norm, weight)
    dfNorm_weight = pd.DataFrame(norm_weight, columns = ['C1', 'C2', 'C3', 'C4', 'C5', 'C6'], index = ['A1', 'A2', 'A3', 'A4', 'A5', 'A6', 'A7','A8','A9','A10','A11','A12','A13','A14','A15','A16','A17','A18','A19','A20','A21','A22','A23','A24','A25','A26','A27','A28','A29','A30','A31','A32','A33','A34'])
    print("\n\nSTEP2: Weighted Normalized : \n")
    print(dfNorm_weight)

    print("\n\nSTEP3: Set Index of Concordance and Discordance:\n")
    index_tables = index_table(norm_weight, "concordance")
    print("index concordance:\n", index_tables)
    index_tables = index_table(norm_weight, "discordance")
    print("\nindex discordance:\n", index_tables)

    cor = concordance(data_arr, weight)
    print("concordance:\n\n", np.array(cor))
    dis = discordance(norm_weight)
    print("\ndiscondance:\n\n", np.array(dis))
    print("===================================")

    print("\n\nSTEP4: Concordance and Discordance Supremacy:\n")
    trs = threshold(cor)
    print("Threshold:", trs)
    Res_cor = dominan_concordance(cor, trs)
    print("concordance:\n", Res_cor)

    trsD = threshold(dis)
    print("\nThreshold:", trsD)
    Res_dis = dominan_discordance(dis, trsD)
    print("discordance:\n", Res_dis)

    print("\n\nSTEP5: MATRIX TOTAL:\n")
    matrix_total = np.array(Res_cor)*np.array(Res_dis)
    dfTotal = pd.DataFrame(matrix_total, columns = ['A1', 'A2', 'A3', 'A4', 'A5', 'A6', 'A7','A8','A9','A10','A11','A12','A13','A14','A15','A16','A17','A18','A19','A20','A21','A22','A23','A24','A25','A26','A27','A28','A29','A30','A31','A32','A33','A34'], index = ['A1', 'A2', 'A3', 'A3', 'A5', 'A6', 'A7','A8','A9','A10','A11','A12','A13','A14','A15','A16','A17','A18','A19','A20','A21','A22','A23','A24','A25','A26','A27','A28','A29','A30','A31','A32','A33','A34'])
    print("Matrix Total:\n", dfTotal)
    

In [ ]:
def main():
    data_arr = dfNew.to_numpy()
    print(dfNew)
    weight = W
    data = np.array(data_arr)

    norm = normalized_decision(data, data.transpose())
    print("\n\nSTEP1: Normalized : \n")
    dfNorm = pd.DataFrame(norm, columns = ['C1', 'C2', 'C3', 'C4', 'C5', 'C6'], index = ['A1', 'A2', 'A3', 'A4', 'A5', 'A6'])
    print(dfNorm)

    norm_weight = weighted_normalized(norm, weight)
    dfNorm_weight = pd.DataFrame(norm_weight, columns = ['C1', 'C2', 'C3', 'C4', 'C5', 'C6'], index = ['A1', 'A2', 'A3', 'A4', 'A5', 'A6'])
    print("\n\nSTEP2: Weighted Normalized : \n")
    print(dfNorm_weight)

    print("\n\nSTEP3: Set Index of Concordance and Discordance:\n")
    index_tables = index_table(norm_weight, "concordance")
    print("index concordance:\n", index_tables)
    index_tables = index_table(norm_weight, "discordance")
    print("\nindex discordance:\n", index_tables)

    cor = concordance(data_arr, weight)
    print("concordance:\n\n", np.array(cor))
    dis = discordance(norm_weight)
    print("\ndiscondance:\n\n", np.array(dis))
    print("===================================")

    print("\n\nSTEP4: Concordance and Discordance Supremacy:\n")
    trs = threshold(cor)
    print("Threshold:", trs)
    Res_cor = dominan_concordance(cor, trs)
    print("concordance:\n", Res_cor)

    trsD = threshold(dis)
    print("\nThreshold:", trsD)
    Res_dis = dominan_discordance(dis, trsD)
    print("discordance:\n", Res_dis)

    print("\n\nSTEP5: MATRIX TOTAL:\n")
    matrix_total = np.array(Res_cor)*np.array(Res_dis)
    dfTotal = pd.DataFrame(matrix_total, columns = ['A1', 'A2', 'A3', 'A4', 'A5', 'A6'], index = ['A1', 'A2', 'A3', 'A4', 'A5', 'A6'])
    print("Matrix Total:\n", dfTotal)

In [ ]:
main()

               C1       C2     C3       C4       C5     C6
alternatif                                                
A1          15900   496.84  33.80   707707  1035784   8.21
A2           1394  4188.52  32.04   751785  2312831   9.59
A3           1058  4119.93  35.63   572837  1092459  13.81
A4           1227   503.14  38.07   369831   277830  14.71
A5            831  4585.97  32.80   818111  1038899  13.38
A6           1338   857.64  32.22  1308214   795314   6.60


STEP1: Normalized : 

          C1        C2        C3        C4        C5        C6
A1  0.986344  0.065933  0.403936  0.357826  0.337754  0.292273
A2  0.086476  0.555836  0.382902  0.380113  0.754181  0.341401
A3  0.065632  0.546734  0.425805  0.289634  0.356235  0.491631
A4  0.076116  0.066769  0.454965  0.186991  0.090596  0.523671
A5  0.051550  0.608580  0.391985  0.413648  0.338770  0.476323
A6  0.083002  0.113813  0.385053  0.661450  0.259340  0.234958


STEP2: Weighted Normalized : 

          C1            C2    

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  app.launch_new_instance()
